# Routing Assignment

Congratulations! You just convinced a group of investors to put money into your startup competing against _Instacart_ for delivering specialty pet supplies for hamster grooming. Immediately upon publishing your website, the City of Dallas signs up to have a pallet of hamster grooming supplies delivered to "1500 Marilla Street, Dallas, TX" (Dallas City Hall--where all office animals are banned except hamsters). In addition, you received an order from Annamelissa Uhumehlemehe (a well-known childish hamster lady) who lives at "11322 Cactus Lane, Dallas, TX". You notice other addresses rolling in--_all in the City of Dallas_ (none in Garland, Grand Prarie, Ft. Worth, etc.). So you better hurry because you don't want to lose out on sales!

You immediately go to the https://openstreetmap.org website and verify that these addresses are good addresses in OpenStreetMap including your warehouse at "5351 Fults Blvd., Dallas, TX" (just off Samuell Blvd., and South Buckner). One of your investors' requirements is that you not spend money on Google products or any other product. _You must use OpenStreetMap_ for your routing application.

Your project is to write an application that optimizes deliveries to 20 addresses at a time in the City of Dallas using OpenStreetMap's API. You may pick 18 other addresses at random but, be careful to validate them in OpenStreetMap. Here are some address details:

- Starting (warehouse) address is "5351 Fults Blvd., Dallas, TX"
- You must include "1500 Marilla Street, Dallas, TX", and "11322 Cactus Lane, Dallas, TX" in your list of delivery addresses
- You may pick 18 other addresses but they also have to process any verified list of OpenStreetMap-valid addresses

Your application must:

- Optimize the route starting from your warehouse address with the shortest distance to complete the route
- Display the destinations and the distance between them (refer to the example in the assignment on Canvas)
- Display a map with the most efficient route displayed (refer to the example in the assignment on Canvas)
- Accept a list of up to 20 delivery addresses in Dallas and generate a new route list and map
- Oh wait! The example on Canvas shows distances in km! We use **miles** here so don't forget to convert!

The rubric for this assignment includes:

- All required components are present and your program works as per the specifications (above)
- Additional innovations may be (should be) added to differentiate your app from your classmates
- Your program is well-commented and demonstrates that you know how it works
- Any and all functions should include _docstrings_ that appear when the "help( )" function is called
- The class presentation is excellent

The cells below instantiate packages that may be of use as a starting place. You may alter them but remember: _you must use OpenStreetMap_ to complete this assignment.

In [4]:
import numpy as np
from typing import List, Tuple, Dict
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow import keras

In [5]:
class MLDeliveryRouter:
    """
    Machine Learning based delivery route optimizer using reinforcement learning
    """
    def __init__(self, learning_rate=0.01, discount_factor=0.95):
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.scaler = StandardScaler()
        self.model = self._build_model()
        self.experience_buffer = []
        
    def _build_model(self):
        """Build neural network model for Q-learning"""
        model = keras.Sequential([
            keras.layers.Dense(128, activation='relu', input_shape=(4,)),
            keras.layers.Dropout(0.2),
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dropout(0.2),
            keras.layers.Dense(32, activation='relu'),
            keras.layers.Dense(1)  # Q-value output
        ])
        model.compile(optimizer='adam', loss='mse')
        return model
    
    def _get_state_features(self, current_location: Tuple[float, float], 
                           remaining_locations: List[Tuple[float, float]]) -> np.ndarray:
        """Extract relevant features for the current state"""
        if not remaining_locations:
            return np.zeros(4)
            
        # Calculate features
        distances = [self._calculate_distance(current_location, loc) 
                    for loc in remaining_locations]
        
        features = [
            np.mean(distances),  # Average distance to remaining locations
            np.std(distances),   # Standard deviation of distances
            min(distances),      # Distance to nearest location
            len(remaining_locations)  # Number of remaining locations
        ]
        
        return np.array(features).reshape(1, -1)
    
    def _calculate_distance(self, loc1: Tuple[float, float], 
                          loc2: Tuple[float, float]) -> float:
        """Calculate Euclidean distance between two points"""
        return np.sqrt((loc1[0] - loc2[0])**2 + (loc1[1] - loc2[1])**2)
    
    def _get_reward(self, distance: float, remaining_locations: int) -> float:
        """Calculate reward based on distance traveled and remaining locations"""
        distance_penalty = -distance / 1000  # Convert to km
        completion_bonus = 100 if remaining_locations == 0 else 0
        return distance_penalty + completion_bonus
    
    def train(self, training_routes: List[List[Tuple[float, float]]], epochs=100):
        """Train the model on historical routes"""
        print("Training ML model...")
        
        for epoch in range(epochs):
            total_loss = 0
            for route in training_routes:
                current_loc = route[0]
                remaining_locs = route[1:]
                
                while remaining_locs:
                    # Get current state
                    state = self._get_state_features(current_loc, remaining_locs)
                    
                    # Try each possible next location
                    best_q = float('-inf')
                    best_next = None
                    
                    for next_loc in remaining_locs:
                        # Predict Q-value for this action
                        next_state = self._get_state_features(next_loc, 
                                    [loc for loc in remaining_locs if loc != next_loc])
                        q_value = self.model.predict(state, verbose=0)[0][0]
                        
                        if q_value > best_q:
                            best_q = q_value
                            best_next = next_loc
                    
                    # Move to best next location
                    distance = self._calculate_distance(current_loc, best_next)
                    reward = self._get_reward(distance, len(remaining_locs) - 1)
                    
                    # Store experience
                    self.experience_buffer.append((state, reward, next_state))
                    
                    # Update current location and remaining locations
                    current_loc = best_next
                    remaining_locs.remove(best_next)
                    
                    # Train on mini-batch
                    if len(self.experience_buffer) >= 32:
                        self._train_on_batch()
                        
            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch + 1}/{epochs} completed")
    
    def _train_on_batch(self, batch_size=32):
        """Train on a batch of experiences"""
        if len(self.experience_buffer) < batch_size:
            return
            
        # Sample random batch
        batch = np.random.choice(len(self.experience_buffer), batch_size, replace=False)
        states = []
        targets = []
        
        for idx in batch:
            state, reward, next_state = self.experience_buffer[idx]
            
            # Calculate target Q-value
            if next_state.any():  # If not terminal state
                next_q = self.model.predict(next_state, verbose=0)[0][0]
                target = reward + self.discount_factor * next_q
            else:
                target = reward
                
            states.append(state[0])
            targets.append(target)
            
        # Train model
        self.model.train_on_batch(np.array(states), np.array(targets))
    
    def optimize_route(self, locations: List[Tuple[float, float]]) -> List[Tuple[float, float]]:
        """Find optimal route using trained model"""
        if len(locations) <= 2:
            return locations
            
        optimized_route = [locations[0]]  # Start with first location
        remaining_locs = locations[1:]
        current_loc = locations[0]
        
        while remaining_locs:
            # Get current state
            state = self._get_state_features(current_loc, remaining_locs)
            
            # Find best next location
            best_q = float('-inf')
            best_next = None
            
            for next_loc in remaining_locs:
                next_state = self._get_state_features(next_loc, 
                            [loc for loc in remaining_locs if loc != next_loc])
                q_value = self.model.predict(state, verbose=0)[0][0]
                
                if q_value > best_q:
                    best_q = q_value
                    best_next = next_loc
            
            # Add best location to route
            optimized_route.append(best_next)
            current_loc = best_next
            remaining_locs.remove(best_next)
        
        return optimized_route
    
    def evaluate_route(self, route: List[Tuple[float, float]]) -> Dict:
        """Evaluate the quality of a route"""
        total_distance = 0
        for i in range(len(route) - 1):
            total_distance += self._calculate_distance(route[i], route[i + 1])
            
        return {
            'total_distance': total_distance,
            'average_distance': total_distance / (len(route) - 1),
            'num_stops': len(route)
        }

In [ ]:
# Initialize router
ml_router = MLDeliveryRouter()

# Train on historical routes
training_routes = [
    [(lat1, lon1), (lat2, lon2), ...],  # Route 1
    [(lat3, lon3), (lat4, lon4), ...],  # Route 2
    # More routes...
]
ml_router.train(training_routes)

# Optimize new route
locations = [(lat1, lon1), (lat2, lon2), ...]  # Your locations
optimized_route = ml_router.optimize_route(locations)

# Evaluate route
metrics = ml_router.evaluate_route(optimized_route)
print(f"Route metrics: {metrics}")